## Escuela de Ingeniería en Computación, ITCR 

## Aprendizaje automático

### Multilayer Perceptron (MLP) utilizando PyTorch


**Profesora: María Auxiliadora Mora**

## Introducción

Otros algoritmos que pueden ser utilizados para realizar regresión no lineal se listan a continuación:

- Multivariate Adaptive Regression Splines (Regresión Spline Adaptativa Multivariante)
- Multilayer Perceptron (Perceptrón Multicapa)



### Multivariate Adaptive Regression Splines

Splines de regresión adaptativa multivariante (MARS) es un método de regresión que modela múltiples no linealidades en los datos mediante funciones bisagra (funciones con una torcedura en ellas)(Friedman 1991).

El algoritmo implica encontrar un conjunto de funciones lineales simples que, en conjunto, den como resultado el mejor rendimiento predictivo. De esta forma, MARS puede lograr un buen desempeño en problemas de regresión complejos con muchas variables de entrada y relaciones no lineales complejas.

Por ejemplo, en la siguiente figura las líneas azules representan los valores predichos (y) en función de x utilizando diferentes algoritmos de regresión. (A) El enfoque de regresión lineal tradicional no captura ninguna no linealidad. (B) Polinomio de grado 2, (C) Polinomio de grado 3, (D) Función escalón que divide x en seis niveles categóricos.

![](../imagenes/nonlinear-comparisons-1.png)

Utilizando MARS, la siguiente figura muestra ejemplos de splines de regresión ajustados de uno (A), dos (B), tres (C) y cuatro (D) nudos. 

![](../imagenes/examples-of-multiple-knots-1.png)


Imágenes de https://bradleyboehmke.github.io/HOML/mars.html

### Multilayer Perceptron (MLP)
Un Perceptrón Multicapa es un algoritmo de aprendizaje supervisado que aprende una función $f(x): \mathbb{R}^{m}-> \mathbb{R}^{o}$ por medio de ajustar los parámetros del modelo usando un conjunto de datos, donde m es el número de dimensiones para la entrada y o las dimensiones de la salida.

Dado un conjunto de características y un objetivo, el perceptrón multicapa puede aprender un aproximador de función no lineal para realizar **clasificación o regresión**.  La figura siguiente muestra un MLP de una capa oculta con salida escalar.

![](../imagenes/MLP.png)

Perceptrón de una capa (Imagen scikit-learn.org)


## MLP Ejemplo

El objetivo de esta sección es implementar un ejemplo básico de regresión utilizando la biblioteca de PyTorch para introducir a los estudiantes en el uso de redes neuronales.

Los datos del ejemplo documentan características de viviendas de una región de Boston. El objetivo del modelo es predecir el valor de las soluciones de vivienda en función de los atributos de la propiedad [5]. 

El ejemplo está basado en [3] y utiliza otras funcionalidades descritas en la lista de referencias.

El proceso de construir un modelo incluye las siguientes etapas: 
- Carga de datos
- Revisión, limpeza de datos, selección de características a utilizar
- Definición del modelo
- Entrenamiento
- Evaluación de modelos
- Uso del modelo seleccionado


In [1]:
# !pip install prettytable

In [2]:
# General libraries
from numpy import vstack
from numpy import sqrt
from numpy import array
from pandas import read_csv, notnull
import numpy as np
import pandas as pd
from prettytable import PrettyTable

# Form Scikit-Learn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# Anomalies detection
from sklearn.ensemble import IsolationForest

# From Pytorch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

### Carga, revisión, limpeza de datos y selección de características a utilizar 

El código para procesar muestras puede ser difícil de mantener. Idealmente, se desea que el conjunto de datos a utilizar esté desacoplado del código de entrenamiento del modelo para logar una mejor legibilidad y modularidad. 

PyTorch proporciona dos primitivas para cargado y uso de datos durante el entrenamiento y evaluación de modelos: torch.utils.data.DataLoader y torch.utils.data.Dataset. Estas clase  permiten manejar conjuntos de datos precargados y disponibles en Internet y datos locales. Dataset almacena las muestras y sus etiquetas correspondientes, y DataLoader envuelve un iterador alrededor de Dataset para permitir un fácil acceso a las muestras.

El conjunto de datos a utilizar es "The Boston Housing Dataset" disponible en https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset.  El conjunto de datos de viviendas de Boston se deriva de la información recopilada por el Servicio del Censo de los EE. UU. sobre viviendas en el área de Boston MA. A continuación se describen las columnas del conjunto de datos:

  - CRIM - per capita crime rate by town
  - ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
  - INDUS - proportion of non-retail business acres per town.
  - CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
  - NOX - nitric oxides concentration (parts per 10 million)
  - RM - average number of rooms per dwelling
  - AGE - proportion of owner-occupied units built prior to 1940
  - DIS - weighted distances to five Boston employment centres
  - RAD - index of accessibility to radial highways
  - TAX - full-value property-tax rate per $ $10,000$
  - PTRATIO - pupil-teacher ratio by town
  - B:  1000(Bk - 0.63)^2  where Bk is the proportion of blacks by town 
  - LSTAT: percentage of lower status of the population
  - MEDV - Median value of owner-occupied homes in dolars 1000's

Un ejemplo claro del racismo existente en algunos conjuntos de datos (observen la columna B).

In [3]:
# Explore data
path = '../../Data/housing.csv'
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = read_csv(path, header=None, names=column_names)
print(data)
print(type(data))
print("Tamaño del dataset", data.shape)
print(data.head(5))


        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...    ...   
501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273.0   
502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273.0   
503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273.0   
504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273.0   
505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273.0   

     PTRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396.90   9.14

In [4]:
# utility functions

# dataset definition
class CSVDataset(Dataset):
    """ 
    load and preprocess the dataset. Extends the functionality of the class Dataset.
    """
    def __init__(self, df):
        """
        The __init__ function is run once when instantiating the Dataset object. 
        :param: df a dataframe with the data to be preprocess. 
        """  
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        
        #print("Tipo de datos", type (self.X))
        #print(self.X)
        
        # Scale the data
        self.transformer = MinMaxScaler().fit(self.X)
        self.X = self.transformer.transform(self.X)
        
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        """
        The __len__ function returns the number of samples in our dataset.
        return: the length of the ndarray X.
        """
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        """
        The __getitem__ function loads and returns 
        a sample from the dataset at the given index idx. 
        :param idx: index.
        return: a list with the value in X[idx] and y[idx].
        """
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        """
        Split the dataset into training and test data.
        :param n_test: training data percentage
        return the training and test data in four vectors (X_train, y_train, X_test, y_test).
        """
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
    
    def scale_data(self, data_lst):
        """
        Scale a data array.
        :param data_array: records.
        return the scaled data using the transform instance of MinMaxScaler.
        """
        data_array = np.array(data_lst)
        data_array = data_array.reshape(1, -1)
        data_array = self.transformer.transform(data_array)
        return(data_array)

    
def prepare_data(boston_df):
    """
    Prepare the dataset.
    :param path: path and name of the file .
    """
    # load the dataset
    dataset = CSVDataset(boston_df)
    #print("dataset.X.len", dataset.__len__())
    #print("dataset.X", dataset.X)
    #print("dataset.y", dataset.y)

    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=10, shuffle=False)
    return train_dl, test_dl, dataset


def evaluate_model(test_dl, model):
    """
    Evaluates the model performance using Mean Squared Error (MSE).
    :param: test_dt, test data.
    :param: model, model to evaluate.
    """
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    # vstack: Stack arrays in sequence vertically (row wise).    
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate the mse and mae
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae


def predict(row, model):
    """
    Make a class prediction for one row of data
    :param: row, data that will be used for prediction.
    :param: model, the model to apply to the data. 
    """
    # scale the row
    row = dataset.scale_data(row)
    
    # convert row to a tensor.
    # Creating a tensor from a list of numpy.ndarrays is extremely slow 
    # then converting the list to a single numpy.ndarray is recomended.
    row = Tensor(np.array([row]))
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

def count_parameters(model):
    """
    Display in a table the model parameters. The model must be a PyTorch DNN model.
    :param: row, data that will be used for prediction.
    :param: model, the model to apply to the data. 
    """
    table = PrettyTable(["Mod name", "Parameters Listed"])
    t_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        t_params+=param
    print(table)
    print(f"Sum of trained paramters: {t_params}")
    return t_params

In [5]:
# model definition
class MLP(Module):
    """
    Class that implements the perceptron, it extends the nn.Module class.
    """
    def __init__(self, n_inputs, n_output, n_layer_1, n_layer_2):
        """
        Defines the model's structure.
        :param: n_inputs, amount of input data.
        :param: n_output, amount of result elements. 
        :param: n_layer_1, number of neurons in layer 1.
        :param: n_layer_2, number of neurons in layer 2.
        """
        super(MLP, self).__init__()

        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, n_layer_1)

        # Initialization: Weights are scaled using a uniform distribution.
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()

        # second hidden layer
        self.hidden2 = Linear(n_layer_1, n_layer_2)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()

        # third hidden layer and output
        self.hidden3 = Linear(n_layer_2, n_output)
        xavier_uniform_(self.hidden3.weight)

    def forward(self, X):
        """
        Forward run of the network using the data in X.
        """
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

In [6]:

def train_model(train_dl, model):
    """
    Train the model using the train data loader (train_dl).
    :param: train_dl, training data accessed via a dataloader.
    :param: model to be trained.
    """
    
    # Define the optimization parameters
    # Mean Squared Error (MSE)
    criterion = MSELoss()
    # Stochastic gradient descent (SGD)
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
    return model

In [7]:
# prepare the data
boston_df = read_csv(path, header=None)

train_dl, test_dl, dataset = prepare_data(boston_df)
print("Cantidad de datos de entrenamiento y prueba", len(train_dl.dataset), len(test_dl.dataset))

# define the network
n_inputs = data.shape[1] -1 
n_layer_1 = 10
n_layer_2 = 8
n_output = 1
model = MLP( n_inputs, n_output, n_layer_1, n_layer_2 )

# train the model
model = train_model(train_dl, model)


Cantidad de datos de entrenamiento y prueba 339 167


In [8]:
# To explore the model parameters
print("Tamaño del dataset (incluyendo el target)", data.shape)
print("==============================================")
print("Cantidad de parámetros del modelo")
count_parameters(model)

Tamaño del dataset (incluyendo el target) (506, 14)
Cantidad de parámetros del modelo
+----------------+-------------------+
|    Mod name    | Parameters Listed |
+----------------+-------------------+
| hidden1.weight |        130        |
|  hidden1.bias  |         10        |
| hidden2.weight |         80        |
|  hidden2.bias  |         8         |
| hidden3.weight |         8         |
|  hidden3.bias  |         1         |
+----------------+-------------------+
Sum of trained paramters: 237


237

### Evaluación del modelo


In [9]:
# evaluate the model using the Mean Squared Error (MSE)
mse, mae = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
print('MAE: %.3f' % (mae))


# make a single prediction
row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
yhat = predict(row, model)
print('Predicted: %.3f' % yhat)

MSE: 11.391, RMSE: 3.375
MAE: 2.510
Predicted: 28.022


El MSE calcula una medida cuantitativa de **qué tan bien puede predecir el modelo la variable objetivo**. Un valor de MSE más bajo indica que el modelo está haciendo predicciones más precisas, mientras que un valor de MSE más alto indica que el modelo está haciendo predicciones menos precisas.

Es importante tener en cuenta que el **MSE es sensible a los valores atípicos**, ya que elevar al cuadrado la diferencia entre los valores predichos y reales puede amplificar el efecto de los valores atípicos. Por lo tanto, es importante tener en cuenta otras métricas de rendimiento, como el error absoluto medio (MAE), si el conjunto de datos contiene valores atípicos.

En resumen, MSE es una métrica de rendimiento ampliamente utilizada en el aprendizaje automático para problemas de regresión.

In [10]:
# Example: how to use dataloaders
print(test_dl)

for i, (inputs, targets) in enumerate(test_dl):
    print(i)
    print("Tamaño de las características", inputs.shape)
    print("Features",inputs)
    print("Targets",targets)


0
Tamaño de las características torch.Size([10, 13])
Features tensor([[3.2794e-02, 0.0000e+00, 7.0088e-01, 0.0000e+00, 4.5267e-01, 4.8668e-01,
         9.2791e-01, 1.0492e-01, 1.7391e-01, 4.1221e-01, 2.2340e-01, 6.0477e-01,
         2.2296e-01],
        [6.1785e-04, 2.0000e-01, 1.0521e-01, 1.0000e+00, 1.1914e-01, 7.8253e-01,
         4.8198e-01, 3.7122e-01, 1.7391e-01, 5.5344e-02, 2.4468e-01, 9.5000e-01,
         3.5320e-02],
        [3.0797e-05, 9.0000e-01, 9.2009e-02, 0.0000e+00, 3.0864e-02, 6.7580e-01,
         1.8435e-01, 5.6177e-01, 0.0000e+00, 1.8702e-01, 2.8723e-01, 9.9450e-01,
         1.6887e-01],
        [1.3080e-03, 0.0000e+00, 2.3644e-01, 0.0000e+00, 1.2963e-01, 4.8055e-01,
         3.8208e-01, 4.1751e-01, 8.6957e-02, 8.7786e-02, 5.6383e-01, 9.8106e-01,
         2.1578e-01],
        [1.0319e-01, 0.0000e+00, 6.4663e-01, 0.0000e+00, 6.4815e-01, 3.7842e-01,
         1.0000e+00, 4.0993e-02, 1.0000e+00, 9.1412e-01, 8.0851e-01, 1.0000e+00,
         6.0348e-01],
        [2.6447e-0

## Regresión eliminando anomalías

In [11]:
# prepare the data
boston_array = boston_df.to_numpy()

# compute the IsolationForest algorithm to detect anomalies.
clf = IsolationForest(n_estimators=100, contamination=.02)
predictions = clf.fit_predict(boston_array)

# data without anomalies.
boston_df_clean = pd.DataFrame(boston_array[np.where(predictions>-1)])

# prepara data before train the model. 
train_dl, test_dl, dataset = prepare_data(boston_df_clean)
print("Cantidad de datos de entrenamiento y prueba", len(train_dl.dataset), len(test_dl.dataset))

# define the network and train the model 
n_inputs = data.shape[1] -1 
n_layer_1 = 10
n_layer_2 = 8
n_output = 1
model = MLP( n_inputs, n_output, n_layer_1, n_layer_2 )

# train the model
model = train_model(train_dl, model)


Cantidad de datos de entrenamiento y prueba 332 163


### Evaluación del modelo después de remover las anomalías  

In [12]:
# evaluate the model using the Mean Squared Error (MSE)
mse, mae = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
print('MAE: %.3f' % (mae))


# make a single prediction
row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
yhat = predict(row, model)
print('Predicted: %.3f' % yhat)

MSE: 16.729, RMSE: 4.090
MAE: 2.876
Predicted: 25.971


#### Referencias:
    
[1] Brownlee, J. (2016). Support Vector Machines for Machine Learning. Recuperado de https://machinelearningmastery.com/support-vector-machines-for-machine-learning/

[2] Maklin, C. (2019). Support Vector Machine Python Example. https://towardsdatascience.com/support-vector-machine-python-example-d67d9b63f1c8

[3] Brownlee, J. (2020). https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

[4] DATASETS & DATALOADERS. Creating a Custom Dataset for your files. Recuperado de https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

[5] Boston Housing Data. https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.names

[6] Brownlee, J. (2021). Weight Initialization for Deep Learning Neural Networks.  https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/

[7] Vapnik, V. The Nature of Statistical Learning Theory. Springer, New York, 1995.

[8] Glorot, X. & Bengio, Y. (2010). Understanding the difficulty of training deep feedforward neural networks. Recuperado de https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

[9] Bishop, C. (2006). Pattern recognition and machine learning. springer. Recuperado de https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf

